In [1]:
#!pip install langdetect


In [2]:
#Importing Libraries
import requests
import pickle
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.robotparser import RobotFileParser
from time import sleep
from urllib.parse import urlparse, urljoin, urlunparse
from urllib.request import urlopen
from collections import Counter
from urllib.error import URLError
from urllib.error import HTTPError
from collections import deque
import ssl
from langdetect import detect, DetectorFactory, lang_detect_exception

ssl._create_default_https_context = ssl._create_unverified_context

### Canonicalization Function

In [3]:
#Canoncalization Function

def canonicalize_url(url):
    """
    Canonicalizes a given URL according to specific rules:
    - 1. Convert the scheme and host to lower case: 
HTTP://www.Example.com/SomeFile.html → http://www.example.com/SomeFile.html
    - 2. Remove port 80 from http URLs, and port 443 from HTTPS URLs: 
http://www.example.com:80 → http://www.example.com
    - 3. Make relative URLs absolute: 
      If you crawl http://www.example.com/a/b.html and find the URL ../c.html, 
      it should canonicalize to http://www.example.com/c.html.
    - 4. Remove the fragment, which begins with #: 
http://www.example.com/a.html#anything → http://www.example.com/a.html
    - 5. Remove duplicate slashes: 
http://www.example.com//a.html → http://www.example.com/a.html
    """
    try:   
    # use the urlparse to parse the URL
        parse_res = urlparse(url) # ParseResult(scheme='https', netloc='www.mphonline.org', path='/worst-pandemics-in-history/', params='', query='', fragment='')

        # 1. Convert scheme and host to lower case
        scheme = parse_res.scheme.lower()
        netloc = parse_res.netloc.lower()

        # 2. Remove default port 80 for HTTP and 443 for HTTPS
        if scheme == 'http' and parse_res.port == 80:
            netloc = parse_res.hostname
        elif scheme == 'https' and parse_res.port == 443:
            netloc = parse_res.hostname

        path = parse_res.path
        # 3. Make relative URLs absolute
        if not path.startswith('/'):
            path = '/' + path

        # 4. Remove the fragment: make the fragment empty
        fragment = ''

        # 5. Remove duplicate slashes
        path = path.replace('//', '/')

        # use urlunparse to reconstruct the URL
        canonicalized_url = urlunparse((scheme, netloc, path, parse_res.params, parse_res.query, fragment))
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None
    return canonicalized_url

test_urls = ['HTTP://www.Example.com/SomeFile.html', 
            'http://www.example.com:80',
            'http://www.example.com/a.html#anything',
            'http://www.example.com//a.html']
for url in test_urls:
    canonicalized_url = canonicalize_url(url)
    print(canonicalized_url)

http://www.example.com/SomeFile.html
http://www.example.com/
http://www.example.com/a.html
http://www.example.com/a.html


### URL Class

In [4]:
# URL Class
class URL:
    def __init__(self, url, inlinks, wave_number, anchor_text):
        self.url = url
        self.inlinks = inlinks
        self.outlinks = set()
        self.wave_number = wave_number
        self.anchor_text = anchor_text
        self.score = 0
        
    def __eq__(self, other):
        return self.url == other.url

    def __hash__(self):
        return hash(self.url)
    
    def getInlinkCount(self):    
        return len(self.inlinks)
    
    def getInlinks(self):    
        return self.inlinks
    
    def getWaveNumber(self):
        return self.wave_number
    
    def getURL(self):
        return self.url
    
    def getScore(self):
        return self.score
    
    def outlinks(self):
        return self.outlinks
    
    def getAnchorTexts(self):
        return self.anchor_text

In [5]:
#Defining variables 
frontier = deque()
visited = set()
seed_urls = ['https://en.wikipedia.org/wiki/COVID-19_pandemic', 'https://www.mphonline.org/worst-pandemics-in-history/', 'https://en.wikipedia.org/wiki/List_of_epidemics',
'https://en.wikipedia.org/wiki/Pandemic']

In [6]:
inlinks_dict = {}
outlinks_dict = {}
robot_parsers = {}

In [7]:
# Canonicalizing seed urls
canonicalized_seed_urls = []
for url in seed_urls:
    canonicalized_seed_urls.append(canonicalize_url(url))
canonicalized_seed_urls

['https://en.wikipedia.org/wiki/COVID-19_pandemic',
 'https://www.mphonline.org/worst-pandemics-in-history/',
 'https://en.wikipedia.org/wiki/List_of_epidemics',
 'https://en.wikipedia.org/wiki/Pandemic']

In [8]:
# Adding seed urls to frontier after making them URL objects 
for url in canonicalized_seed_urls:
    wave_number = 0
    inlinks = set()
    anchor_text =''
    frontier.append(URL(url, inlinks, wave_number, anchor_text))

print(frontier[0].url)    

https://en.wikipedia.org/wiki/COVID-19_pandemic


### Crawling code along with robot parser handling

In [9]:
def can_fetch(url):
    parsed_url = urlparse(url)
    robot_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"

#     if robot_url not in robot_parsers:
#         parser = RobotFileParser()
#         parser.set_url(robot_url)
#         try:
#             parser.read()
#             robot_parsers[robot_url] = parser
#         except URLError as e:
#             print(f"Error reading {robot_url}: {e}")
#             return False
    if robot_url not in robot_parsers:
        parser = RobotFileParser()
        parser.set_url(robot_url)
        try:
            # Use urlopen with timeout to fetch the robots.txt
            response = urlopen(robot_url, timeout=1)
            # Read and parse the robots.txt content
            content = response.read().decode('utf-8')
            parser.parse(content.splitlines())
            robot_parsers[robot_url] = parser
        except HTTPError as e:
            # Handle HTTP errors
            if e.code == 403:
                print(f"Access to {robot_url} is forbidden: {e}")
            elif e.code == 404:
                print(f"{robot_url} was not found: {e}")
            else:
                print(f"HTTP Error accessing {robot_url}: {e}")
            # Decide how to proceed in case of HTTP errors. Here we choose to allow fetching.
            return False    

        except URLError as e:
            if isinstance(e.reason, TimeoutError):
                print(f"TimeoutError: Fetching {robot_url} timed out.")
            else:
                print(f"URLError: Error reading {robot_url}: {e.reason}")
            return False
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return False

    return robot_parsers[robot_url].can_fetch("*", url)

In [10]:
def fetch_content(url):
    if not can_fetch(url):
        print(f"Blocked by robots.txt: {url}")
        return None
    try:
        response = requests.get(url, timeout=1)
    except requests.exceptions.Timeout:
    # Handle the timeout exception
        print("The request timed out")  
        return None
    except requests.exceptions.SSLError as e:
        print(f"SSL Error: {e}")
        return None
    except requests.exceptions.RequestException as e:
    # Handle other potential exceptions, such as a connection error
        print(f"An error occurred: {e}")
        return None
    content_type = response.headers.get('Content-Type', '')
    language = response.headers.get('Content-Language', '')
    return response.text, content_type, language

In [11]:
def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.title.text if soup.title else ""
    paragraphs = [p.text for p in soup.find_all('p')]
    outlinks = [(urljoin(url, a.get('href')), a.get_text()) for a in soup.find_all('a', href=True)]
    return title, paragraphs, outlinks

In [12]:
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except lang_detect_exception.LangDetectException:
        return None

In [13]:
def crawl_url(url):
    if not can_fetch(url):
        return None
    sleep(1) 
    result = fetch_content(url)

    if result is None:
        return None
    else:
        content, content_type, language = result
    if 'text/html' not in content_type or 'text/htm' not in content_type:
        return None
#     if 'en' not in language:
#         return None
    if content:
        title, paragraphs, outlinks = parse_html(content)
#         if (not len(paragraphs) == 0 and detect_language(paragraphs[0]) == 'en'):
        return {'title': title, 'paragraphs': paragraphs, 'outlinks': outlinks}
#         if title and detect_language(paragraphs[0]) == 'en':
#             return {'title': title, 'paragraphs': paragraphs, 'outlinks': outlinks}
#         else:
#             return None

In [14]:
# Function to save html data
def saveContents(url, output, file_path):
    title = output['title']
    paragraph = output['paragraphs'] 
    output_str = f'<DOC>\n<DOCNO>{url}</DOCNO>\n' + \
                  f'<HEAD>{title}</HEAD>\n' + \
                  f'<TEXT>\n{paragraph}\n</TEXT>\n</DOC>\n'
    
    # Append the formatted string to the file
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write(output_str)

### Setting Variables

In [15]:
#Defining variables
NUMBER_OF_DOCUMENTS_TO_CRAWL = 30000
OUTPUT_FILENAME = './output-crawled/result'
BATCH_SIZE = 500
INLINKS_FILENAME = './output-crawled/inlinks.pkl'
OUTLINKS_FILENAME = './output-crawled/outlinks.pkl'
KEYWORDS = ["pandemic", "covid", "lockdowns", "COVID-19", "vaccines", "Wuhan"]
SAVED_STATE = './output-crawled/crawler-saved-state.pkl'

In [16]:
def updateFrontier(result, frontier, filename):
    new_frontier = deque()
    anchor_texts = []
    for url, output in result.items():
        #Take one url at a time from outlinks
        #Canonicalize, check if the url is present in the frontier or visited, update the inlink count
        #Update the outlinks
        saveContents(url.url, output, filename)
        for outlink, anchor_text in output['outlinks'][:100]:
            outlink = canonicalize_url(outlink)
            outlinkURL = URL(outlink, url.url, url.wave_number+1, anchor_text)
            if outlinkURL.url not in url.outlinks:
                url.outlinks.add(outlinkURL.url)
            #if outlinkURL  in visited:
                
            new_frontier.append(outlinkURL)    
        if url.anchor_text != '':
            anchor_texts.append(url.anchor_text)
            
    return new_frontier, anchor_text, frontier

In [17]:
def calculateScores(frontier, anchor_texts):
    anchor_text_counts = Counter(anchor_texts)
    for i in range(len(frontier)):
        frontier[i].score += frontier[i].getInlinkCount()
        if len(anchor_texts) != 0:
            if frontier[i].anchor_text in anchor_texts:
                frontier[i].score += 10
            if frontier[i].anchor_text in KEYWORDS:
                frontier[i].score += 20

In [18]:
def sortDequeScoreBased(url_deque):
    sorted_list = sorted(list(url_deque), key=lambda url: url.score, reverse=True)
    sorted_deque = deque(sorted_list)
    return sorted_deque

In [19]:
def save_inlinks(state_file, inlinks):
    with open(state_file, 'wb') as f:
        pickle.dump(inlinks, f)
    print("Inlinks saved.")

def save_outlinks(state_file, outlinks):
    with open(state_file, 'wb') as f:
        pickle.dump(outlinks, f)
    print("Outlinks saved.")

In [20]:
def load_inlinks(state_file):
    try:
        with open(state_file, 'rb') as f:
            inlinks = pickle.load(f)
        print("Inlinks loaded successfully.")
        return inlinks
    except FileNotFoundError:
        print(f"No state file found for inlinks at {state_file}.")
        return {}

def load_outlinks(state_file):
    try:
        with open(state_file, 'rb') as f:
            outlinks = pickle.load(f)
        print("Outlinks loaded successfully.")
        return outlinks
    except FileNotFoundError:
        print(f"No state file found for outlinks at {state_file}.")
        return {}

In [21]:
def url_ends_with_blacklisted_extension(url):
    blacklisted_extensions = [".mp4", ".exe", ".zip", ".wmv", ".jpg", ".pdf"]
    url_lower = url.lower()
    return any(url_lower.endswith(ext.lower()) for ext in blacklisted_extensions)

In [22]:
def sliceDictionary(result_dict, start_index):
    sliced_dictionary = dict(list(result_dict.items())[start_index:])
    return sliced_dictionary

In [23]:
def save_crawler_state(state_file, file_counter, frontier, visited, number_of_documents_crawled, wave):
    state = {
        'file_counter': file_counter,
        'frontier': frontier,
        'visited': visited,
        'number_of_documents_crawled': number_of_documents_crawled,
        'wave': wave
    }
    with open(state_file, 'wb') as f:
        pickle.dump(state, f)
    print("Crawler state saved.")


In [24]:
def load_crawler_state(state_file):
    try:
        with open(state_file, 'rb') as f:
            state = pickle.load(f)
        print("Crawler state loaded successfully.")
        return state
    except FileNotFoundError:
        print(f"No state file found at {state_file}. Starting from scratch.")
        return None

In [25]:
# Checks if the url/outlink is present in frontier, visited and updates the inlinks 
def updateInlinks(frontier, visited, next_frontier, inlink, url):
    isPresent = False
    for urlObject in frontier:
        if urlObject.url == url:
            isPresent = True
            urlObject.inlinks.add(inlink)
    for urlObject in next_frontier:
        if urlObject.url == url:
            isPresent = True
            urlObject.inlinks.add(inlink) 
    for urlObject in visited:
        if urlObject.url == url:
            isPresent = True
            urlObject.inlinks.add(inlink) 
                
    if url not in inlinks_dict.keys():
        inlinks_dict[url] = set(inlink)
    else:
        inlinks_dict[url].add(inlink)
    return isPresent, frontier, next_frontier, visited        

In [26]:
def updateNextFrontier(next_frontier, frontier, visited, results, anchor_texts):
    #save contents from result
    #add the outlinks- after canonicalization, urlobject to the next_frontier
    #update the inlinks in frontier new_frontier and the visited
    #add outlinks, anchor_text
    for url, output in results.items():
        #saveContents(url.url, output, filepath)
        if len(output['outlinks']) > 100:
            outlink_selection =  output['outlinks'][:100]
        else:
            outlink_selection =  output['outlinks']
        for outlink, anchor_text in outlink_selection:
            outlink_canonicalized = canonicalize_url(outlink)
            if outlink_canonicalized == None:
                continue
            if 'wikipedia' in outlink_canonicalized:
                if 'en.wikipedia' not in outlink_canonicalized:
                    continue
            isPresent, frontier, next_frontier, visited = updateInlinks(frontier,visited, next_frontier, url.url, outlink_canonicalized)
            if not isPresent:
                if url_ends_with_blacklisted_extension(outlink_canonicalized):
                    continue
                inlink_list = set()
                inlink_list.add(url.url)
                outlinkURLObject = URL(outlink_canonicalized, inlink_list, url.wave_number+1, anchor_text)
                next_frontier.append(outlinkURLObject)
            if outlink_canonicalized not in url.outlinks:
                url.outlinks.add(outlink_canonicalized)
            if url.url not in outlinks_dict.keys():
                outlinks_dict[url.url] = set(outlink_canonicalized)
            else:
                outlinks_dict[url.url].add(outlink_canonicalized)
                    
        if url.anchor_text != '' and url.anchor_text not in anchor_texts:
            anchor_texts.append(url.anchor_text)
        visited.add(url)    
    return next_frontier, frontier, visited, anchor_texts        
        

In [27]:
def crawl_url_with_timeout(url):
    try:
        sleep(1)
        return crawl_url(url)
        
    except Exception as e:
        print(f"Error crawling {url}: {e}")
        return None

In [28]:
def checkFileValidity(output):
    if len(output['paragraphs']) == 0:
        return False
#     elif output['title'] == 'Access Denied':  
#         return False
    return True    

In [29]:
import concurrent.futures

def frontierCrawl(frontier, visited, resumeFailedCrawl = False):
    #Wave - is the current values in frontier, when the frontier is completely cralwed the wave is completed
    # the next wave starts when we update frontier with next_frontier
    #To update the file names       
    file_counter = 1
    wave = 0
    #Counting the number of urls that is cralwed and saved
    number_of_documents_crawled = 0
    # Runs until the frontier is empty
  
    while frontier:
        # Used to calculate score for next set of urls(next_frontier) using the current/previous url anchor texts(frontier)
        anchor_texts = []
        results_saved = 0  
        # To store the next set of froniters
        next_frontier = deque()
        
        if resumeFailedCrawl == True:
            state = load_crawler_state(SAVED_STATE)
            if state is not None:
                file_counter = state['file_counter']
                frontier = state['frontier']
                visited = state['visited']
                number_of_documents_crawled = state['number_of_documents_crawled']
                wave = state['wave']
                    
        # to store results from crawling the url
        results = {}
        
        # Create a thread pool
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            # Submit tasks to the thread pool
            future_to_url = {executor.submit(crawl_url_with_timeout, url.url): url for url in frontier}

            for future in concurrent.futures.as_completed(future_to_url):
                url = future_to_url[future]

                try:
                    output = future.result(timeout = 1)
                    if output is not None:
                        print(f"Crawled Successfully: {url.url}")
                        results[url] = output
                        number_of_documents_crawled += 1
                        
                        if number_of_documents_crawled % 10 == 0:
                            print("Number of documents crawled:" + str(number_of_documents_crawled))
                        
                        saveContents(url.url, output, OUTPUT_FILENAME + str(file_counter) + '.txt')
                        if number_of_documents_crawled % BATCH_SIZE == 0:
                            #required_dictionary = sliceDictionary(results, (BATCH_SIZE*results_saved))                               
                            print(f"BATCH {file_counter} Saved")
                            file_counter += 1
                            #results_saved += 1
                                
                    # True when we reach the number of documents to be crawled
                        if number_of_documents_crawled == NUMBER_OF_DOCUMENTS_TO_CRAWL:
#                             if not number_of_documents_crawled % BATCH_SIZE == 0:
#                                 required_dictionary = sliceDictionary(results, (BATCH_SIZE*results_saved))
#                                 for url, output in required_dictionary.items():
#                                     saveContents(url.url, output, OUTPUT_FILENAME + str(file_counter) + '.txt')
#                                 file_counter += 1
#                                 results_saved += 1
#                             next_frontier, frontier, visited, anchor_texts = updateNextFrontier(
#                                 next_frontier, frontier, visited, results, anchor_texts)
                            save_inlinks(INLINKS_FILENAME, inlinks_dict)
                            save_outlinks(OUTLINKS_FILENAME, outlinks_dict)
                            return        
    
                except TimeoutError:
                    print(f"Timeout crawling {url.url}")            
                except Exception as exc:
                    print(f"Error crawling {url.url}: {exc}")

            
        

        # Process executed when the frontier is completely crawled - updates with next frontier
        next_frontier, frontier, visited, anchor_texts = updateNextFrontier(
            next_frontier, frontier, visited, results, anchor_texts)
        save_inlinks(INLINKS_FILENAME, inlinks_dict)
        save_outlinks(OUTLINKS_FILENAME, outlinks_dict)
        results = {}
        frontier = next_frontier.copy()
        print('Frontier Updated')
        
        # Calculates the scores for the next set of URLs
        calculateScores(frontier, anchor_texts)
        
        # Arranges based on the Priority based on the score
        frontier = sortDequeScoreBased(frontier)

        print(f"Wave {wave} Completed")
        
        wave += 1 
        
        save_crawler_state(SAVED_STATE, file_counter, frontier, visited, number_of_documents_crawled, wave)
        
        print(f"Number of links in Wave {wave}: {len(next_frontier)}")
        
        
        

In [30]:
frontierCrawl(frontier, visited, resumeFailedCrawl = False)

Access to https://www.mphonline.org/robots.txt is forbidden: HTTP Error 403: Forbidden
Crawled Successfully: https://en.wikipedia.org/wiki/List_of_epidemics
Crawled Successfully: https://en.wikipedia.org/wiki/Pandemic
Crawled Successfully: https://en.wikipedia.org/wiki/COVID-19_pandemic
Inlinks saved.
Outlinks saved.
Frontier Updated
Wave 0 Completed
Crawler state saved.
Number of links in Wave 1: 57
Crawled Successfully: https://en.wikipedia.org/wiki/Main_Page
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia:Contents
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia:About
Crawled Successfully: https://en.wikipedia.org/wiki/Portal:Current_events
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia:Contact_us
Crawled Successfully: https://en.wikipedia.org/wiki/Help:Contents
Crawled Successfully: https://en.wikipedia.org/wiki/Help:Introduction
Number of documents crawled:10
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia:Community_portal
Cr

An unexpected error occurred: 'utf-8' codec can't decode byte 0xf6 in position 66: invalid start byte
Crawled Successfully: https://www.wikidata.org/wiki/Q506680
Crawled Successfully: https://www.wikidata.org/wiki/Q44512
Crawled Successfully: https://id.loc.gov/authorities/sh85044370
https://id.ndl.go.jp/robots.txt was not found: HTTP Error 404: Not Found
Access to https://hls-dhs-dss.ch/robots.txt is forbidden: HTTP Error 403: Forbidden
Crawled Successfully: https://thes.bncf.firenze.sbn.it/termine.php?id=21395
Crawled Successfully: https://catalogue.bnf.fr/ark:/12148/cb11949129v
https://datos.bne.es/robots.txt was not found: HTTP Error 404: Not Found
Crawled Successfully: https://scholia.toolforge.org/Q44512
Crawled Successfully: https://wikidocumentaries-demo.wmcloud.org/Q44512
Number of documents crawled:110
Crawled Successfully: https://babelnet.org/synset?id=bn:00031153n&lang=EN
Crawled Successfully: https://locator-tool.toolforge.org/
Crawled Successfully: https://en.wikipedia.o

Crawled Successfully: https://en.wikipedia.org/wiki/Bacteria
Crawled Successfully: https://en.wikipedia.org/wiki/Mediterranean_Basin
Crawled Successfully: https://en.wikipedia.org/wiki/Near_East
Crawled Successfully: https://en.wikipedia.org/wiki/Northern_Europe
Number of documents crawled:200
Crawled Successfully: https://en.wikipedia.org/wiki/Justinian_I
Crawled Successfully: https://en.wikipedia.org/wiki/Procopius
Crawled Successfully: https://en.wikipedia.org/wiki/John_the_Deacon_(Neapolitan_historian)
Crawled Successfully: https://en.wikipedia.org/wiki/Late_antiquity
Crawled Successfully: https://en.wikipedia.org/wiki/Naples_Plague
Crawled Successfully: https://en.wikipedia.org/wiki/Evagrius_Scholasticus
Crawled Successfully: https://en.wikipedia.org/wiki/John_of_Ephesus
Crawled Successfully: https://en.wikipedia.org/wiki/Gregory_of_Tours
Crawled Successfully: https://en.wikipedia.org/wiki/Paul_the_Deacon
Crawled Successfully: https://en.wikipedia.org/wiki/Theophanes_the_Confessor

Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Attalus_I
Crawled Successfully: https://commons.wikimedia.org/wiki/Category:Attalus_I
Number of documents crawled:300
Crawled Successfully: https://en.wikisource.org/wiki/Portal:Attalus_I_of_Pergamum
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:George_Griffith
Crawled Successfully: https://commons.wikimedia.org/wiki/Category:George_Chetwynd_Griffith
Crawled Successfully: https://en.wikisource.org/wiki/Author:George_Chetwynd_Griffiths-Jones
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia_talk:Today%27s_featured_article/March_2024
Crawled Successfully: https://en.wikipedia.org/wiki/Template:FApages
Crawled Successfully: https://en.wikipedia.org/wiki/Template_talk:FApages
Crawled Successfully: https://en.wikipedia.org/wiki/Wikipedia_talk:About_Today%27s_featured_article
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Safe_House_Black_History_Museum
Crawled Successfully: https://commons.wikimedia.or

Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Theophanes_the_Confessor
Crawled Successfully: https://commons.wikimedia.org/wiki/Category:Theophanes_the_Confessor
Crawled Successfully: https://en.wikipedia.org/wiki/Divine_retribution
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Divine_retribution
Crawled Successfully: https://en.wikipedia.org/wiki/Medieval_Greek
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Medieval_Greek
Number of documents crawled:400
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Bubonic_plague
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Bubo
Crawled Successfully: https://commons.wikimedia.org/wiki/Category:Bubo_(pathology)
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Chronicle_of_Seert
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Pseudo-Zacharias_Rhetor
Crawled Successfully: https://commons.wikimedia.org/wiki/File:Paul_F%C3%BCrst,_Der_Doctor_Schnabel_von_Rom_(coloured_version).png
Crawl

...

Crawled Successfully: https://en.wikipedia.org/wiki/Lethal_(disambiguation)
Crawled Successfully: https://en.wikipedia.org/wiki/Diseases
Crawled Successfully: https://en.wikipedia.org/wiki/Chemical_weapons
Crawled Successfully: https://en.wikipedia.org/wiki/Nuclear_fallout
Crawled Successfully: https://en.wikipedia.org/wiki/Toxic
Number of documents crawled:29630
Crawled Successfully: https://en.wikipedia.org/wiki/Biological_weapons
Crawled Successfully: https://en.wikipedia.org/wiki/Overpressure
Crawled Successfully: https://en.wikipedia.org/wiki/Process_safety
Crawled Successfully: https://en.wikipedia.org/wiki/Microbiologist
Crawled Successfully: https://en.wikipedia.org/wiki/Thermal_radiation
Crawled Successfully: https://en.wikipedia.org/wiki/Food_scientist
Crawled Successfully: https://en.wikipedia.org/wiki/Lethal_dose
Crawled Successfully: https://en.wikipedia.org/wiki/Stopping_power
Crawled Successfully: https://www.wikidata.org/wiki/Q1720706
Crawled Successfully: https://en.wi

Crawled Successfully: https://en.wikipedia.org/wiki/Talk:1923_Greek_legislative_election
The request timed out
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:1981_enlargement_of_the_European_Communities
The request timed out
The request timed out
The request timed out
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:2015_Greek_bailout_referendum
Number of documents crawled:29740
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Academy
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:2018_Attica_wildfires
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:The_Acharnians
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Achilles
Crawled Successfully: https://en.wikipedia.org/wiki/Talk:Acropolis
Crawled Successfully: https://en.wikipedia.org/wiki/George_II_of_Greece
Crawled Successfully: https://en.wikipedia.org/wiki/User:Therealscorp1an
The request timed out
Crawled Successfully: https://commons.wikimedia.org/wiki/User:Helix84
Crawled Succes

Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=K
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=L
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=M
Number of documents crawled:29820
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=O
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=N
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=P
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=Q
Crawled Successfully: https://en.wikipedia.org/wiki/Category:Mid-importance_Classical_Greece_and_Rome_articles?from=S
Crawled Successfully: 

Crawled Successfully: https://en.wikipedia.org/wiki/Fexinidazole
Crawled Successfully: https://en.wikipedia.org/wiki/Pentamidine
Number of documents crawled:29900
Crawled Successfully: https://en.wikipedia.org/wiki/Suramin
Crawled Successfully: https://en.wikipedia.org/wiki/Trypanosomiasis_vaccine
Crawled Successfully: https://en.wikipedia.org/wiki/Opportunistic_infections
Crawled Successfully: https://en.wikipedia.org/wiki/Molecular_Koch%27s_postulates
Crawled Successfully: https://en.wikipedia.org/wiki/Falkow%27s_criteria
Crawled Successfully: https://en.wikipedia.org/wiki/Nucleic_acids
Crawled Successfully: https://en.wikipedia.org/wiki/Watson_and_Crick
Crawled Successfully: https://en.wikipedia.org/wiki/Genetic_information
Crawled Successfully: https://en.wikipedia.org/wiki/Relapse
Crawled Successfully: https://doi.org/10.1093%2Fcid%2F10.Supplement_2.S274
Number of documents crawled:29910
Crawled Successfully: https://pubmed.ncbi.nlm.nih.gov/3055197
Crawled Successfully: https://do

The request timed out
The request timed out
The request timed out
The request timed outThe request timed out

The request timed out
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
An unexpected error occurred: 'utf-8' codec can't decode byt

In [ ]:
# # Incase of failure
# inlinks_dict = load_inlinks(INLINKS_FILENAME)
# outlinks_dict = load_outlinks(OUTLINKS_FILENAME)
# frontierCrawl(frontier, visited, True)